In [ ]:
%env USERNAME=<username>

**Warning:** Remember that for interacting with EDI Big Data Stack you must be authenticated at the system using kinit command. For more information, read the documentation at [Authenticating with Kerberos](https://docs.edincubator.eu/big-data-stack/basic-concepts.html#authenticating-with-kerberos).

In [ ]:
%%bash
kinit -kt ~/work/$USERNAME.service.keytab $USERNAME@EDINCUBATOR.EU

# MapReduce & YARN
EDI Big Data Stack provides the MapReduce implementation over YARN. We have created a minimal example, based on [Yelp dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset/version/6) that shows how to count how many Yelp businesses are in each USA state, and how to submit this MapReduce to EDI Big Data Stack.

Yelp dataset is available for every user at */samples/yelp*. Open a terminal at Jyupyter Notebook and execute the following for inspecting the data:

In [ ]:
%%bash
hdfs dfs -ls -h /samples/yelp

You can find all examples at *~/work/examples* directory at your Jupyter Lab instance. A this section we will explain how to launch a typical map-reduce work over Yelp dataset. You can find this example project at *mrexample* folder. The relevant files at this project are *BusinessPerStateCount.java* and *pom.xml*. Later, we are going to inspect *BusinessPerStateCount.java* file.

The *BusinessPerStateCount.java* file contains the unique and main class of this MapReduce job, the BusinessPerStateCount class, and two inner classes, *RowTokenizerMapper* and *StateSumReducer*.

## RowTokenizerMapper

```java
public static class RowTokenizerMapper extends Mapper<Object, Text, Text, IntWritable> {

private final static IntWritable one = new IntWritable(1);

public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
      // Extract state using opencsv library
      CSVReader reader = new CSVReader(new StringReader(value.toString()));
      String[] line;

      while ((line = reader.readNext()) != null) {
          // Check that current line is not CSV's header
          if (!line.equals("state")) {
              // Write "one" for current state to context
              context.write(new Text(line[5]), one);
          }
      }
  }
}
```

The *RowTokenizerMapper* class represents the mapper of our job. Its definition is very simple, as it only extends the base *Mapper* class, receiving a tuple formed by a key of type *Object* and a value of type Text as input, and generating a tuple formed by a key of type Text and a value of type IntWritable as output.

The map method processes the input and generates the output that is passed passed to the reducer. In this function, we take the value, representing the state where the business is, and writes a tuple formed by the state as key, and a “one” as a value. This allow us grouping all appearances of a state in the reducer stage.

## StateSumReducer

```java
public static class StateSumReducer extends Reducer<Text, IntWritable, Text, IntWritable> {

  private IntWritable result = new IntWritable();

  public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
      int sum = 0;
      // For each state coincidence, +one
      for (IntWritable val : values) {
          sum += val.get();
      }
      result.set(sum);

      // Return the state and the number of appearances.
      context.write(key, result);
  }
}
```

The *StateSumReducer* class represents the reducer stage of our job. Similar to the mapper, its definition states that it receives a tuple formed by key of type Text and a value of type IntWritable (generated by the mapper) and produces a tuple formed by key of type Text and a value of type IntWritable.

The reduce function executes the logic of the reducer stage. It receives a key of type text and an *Iterable* of *IntWritables*. The MapReduce framework groups all tuples generated at *RowTokenizerMapper* by its keys, and stores the values for each key in a collection of *Iterable\<IntWritable\>* type. In the case of our example, for each value in the *Iterable* collection, we iterate the collection incrementing the counter obtaining the total count per key.

## main

Finally, the *main* method of the *BusinessPerStateCount* class, which creates and configures the job, has the following code:

```java
public static void main(String [] args) throws IOException, ClassNotFoundException, InterruptedException {
  Configuration conf = new Configuration();
  Job job = Job.getInstance(conf, "state count");
  job.setJarByClass(BusinessPerStateCount.class);

  job.setMapperClass(RowTokenizerMapper.class);
  job.setReducerClass(StateSumReducer.class);

  job.setOutputKeyClass(Text.class);
  job.setOutputValueClass(IntWritable.class);

  FileInputFormat.addInputPath(job, new Path(args[0]));
  FileOutputFormat.setOutputPath(job, new Path(args[1]));

  System.exit(job.waitForCompletion(true) ? 0 : 1);
}
```

In the main method, the MapReduce job is configured. Concretely, this examples sets the mapper and reducer classes, the output key and value classes and the input and output directories (taken from the CLI when launching the job).

## pom.xml 

The *pom.xml* file compiles the project and generates the jar that we need to submit to EDI Big Data Stack.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
       xsi:schemaLocation="http://maven.apache.org/POM/4.0.0 http://maven.apache.org/xsd/maven-4.0.0.xsd">
  <modelVersion>4.0.0</modelVersion>

  <groupId>eu.edincubator.stack.examples</groupId>
  <artifactId>mr-example</artifactId>
  <version>1.0-SNAPSHOT</version>

  <build>
      <plugins>
          <plugin>
              <artifactId>maven-assembly-plugin</artifactId>
              <configuration>
                  <archive>
                      <manifest>
                          <mainClass>eu.edincubator.stack.examples.mr.BusinessPerStateCount</mainClass>
                      </manifest>
                  </archive>
                  <descriptorRefs>
                      <descriptorRef>jar-with-dependencies</descriptorRef>
                  </descriptorRefs>
              </configuration>
          </plugin>
      </plugins>
  </build>

  <dependencies>
      <dependency>
          <groupId>org.apache.hadoop</groupId>
          <artifactId>hadoop-mapreduce-client-core</artifactId>
          <version>${hadoop.version}</version>
          <scope>provided</scope>
      </dependency>
      <dependency>
          <groupId>org.apache.hadoop</groupId>
          <artifactId>hadoop-common</artifactId>
          <version>${hadoop.version}</version>
          <scope>provided</scope>
      </dependency>
      <dependency>
          <groupId>com.opencsv</groupId>
          <artifactId>opencsv</artifactId>
          <version>4.1</version>
      </dependency>
  </dependencies>

  <properties>
      <hadoop.version>2.7.3</hadoop.version>
  </properties>
</project>
```

This file contains two important parts. The fist one, is the *\<build\>* block. This block stablished how the jar is going to be built. In our case, we have choose to create a "fat jar" including the third party dependencies (*com.opencsv* library). On the other hand, the *\<dependencies\>* block contains the dependencies of our project. It is important to import the correct version of the libraries. For more information check [Tools and versions](https://docs.edincubator.eu/big-data-stack/architecture.html#tools-and-versions).

## Compiling and submitting the job

First, from a JupyterLab terminal, you must create the java package:

In [ ]:
%%bash
cd ~/work/examples/mrexample
mvn clean compile assembly:single

In [ ]:
%%bash
cd ~/work/examples/mrexample
yarn jar target/mr-example-1.0-SNAPSHOT-jar-with-dependencies.jar /samples/yelp/yelp_business/yelp_business.csv /user/$USERNAME/state-count-output

If the job is successfully executed, the result is written to the `/user/<username>/state-count-output` directory. In case of any problem during its execution, the error will be printed to the console. For further details about the job, you can check the ResourceManager UI at [https://edi-master.novalocal:8443/gateway/hdp/yarnuiv2].

TODO: update URL

Finally, if you check the output directory, you will see the result of the job as a part-r-00000 file. The execution of this job generated a single file because only one reducer is executed. However, the output could be split into different files if more reducers were required to perform the job.

Then, we can list the files inside the output directory and print, directly to the console, the contents of the generated file. The `-cat` parameter shows the contents of the file, showing the number of businesses for each USA state obtained as the result of the map reduce job.

In [ ]:
%%bash
hdfs dfs -ls /user/$USERNAME/state-count-output

In [ ]:
%%bash
hdfs dfs -cat /user/$USERNAME/state-count-output/part-r-00000